# 03. Chạy Thí nghiệm & Thu thập Kết quả

**Mục tiêu:** Đây là notebook "chính" của dự án. Nó sẽ:

1.  Định nghĩa một danh sách các vấn đề TSP (ví dụ: `berlin52`, `burma14`, `eil76`).
2.  Gọi **tất cả 10 thuật toán** từ thư mục `/algorithms/`.
3.  Chạy từng thuật toán trên từng vấn đề.
4.  Đo lường **thời gian chạy** (`time`) và **chi phí lộ trình** (`cost`).
5.  Tính toán **khoảng cách %** (`gap %`) so với giải pháp tối ưu (nếu có).
6.  Lưu tất cả dữ liệu thô vào một file `results/results.csv` duy nhất bằng `pandas`.

Notebook này được thiết kế để chạy trong thời gian dài (có thể mất vài phút đến vài giờ tùy thuộc vào cấu hình).

**LƯU Ý QUAN TRỌNG:** Các thuật toán Exact (`Brute Force`, `Held-Karp`) sẽ được **bỏ qua (skipped)** một cách có chủ đích trên các vấn đề lớn (`N > 10` hoặc `N > 21`) để tránh thời gian chạy vô tận.

## 1. Thiết lập & Imports

Chúng ta sẽ import `pandas` (để xử lý data) và `time` (để đo thời gian) cùng với tất cả các module thuật toán và tiện ích của chúng ta.

In [1]:
import sys
import os
import time
import pandas as pd
import numpy as np

# --- Cấu hình sys.path --- 
# Thêm thư mục gốc của dự án (cao hơn 'notebooks' một cấp) vào sys.path
# để chúng ta có thể import các module 'utils' và 'algorithms'
project_root = os.path.normpath(os.path.join(os.path.abspath(os.path.dirname("__file__")), '..'))
if project_root not in sys.path:
    sys.path.append(project_root)
    print(f"Đã thêm '{project_root}' vào sys.path")
else:
    print(f"'{project_root}' đã có trong sys.path")
    
# --- Import Utils --- 
from utils import data_loader
from utils import evaluator

# --- Import 10 Thuật toán --- 
# Exact
from algorithms import brute_force
from algorithms import held_karp
# Heuristics Cổ điển
from algorithms import nearest_neighbor
from algorithms import nearest_insertion
# Xấp xỉ
from algorithms import christofides
# Cải tiến
from algorithms import two_opt
# Metaheuristics
from algorithms import simulated_annealing
from algorithms import genetic_algorithm
from algorithms import ant_colony
from algorithms import tabu_search

print("Tất cả các module đã được import thành công.")

Đã thêm 'c:\Desktop\Documents\GitHub\tsp-algorithm-analysis' vào sys.path
Tất cả các module đã được import thành công.


## 2. Cấu hình Thí nghiệm

Định nghĩa các vấn đề chúng ta muốn chạy và các siêu tham số (hyperparameters) cho các thuật toán Metaheuristic.

In [2]:
# --- TỰ ĐỘNG QUÉT (VÀ LỌC) CÁC VẤN ĐỀ ĐỂ CHẠY ---
DATA_DIR = '../data'
RESULTS_FILE = '../results/results.csv'

# Nơi chứa các file .tsp
search_dirs = [
    os.path.join(DATA_DIR, 'tsplib'),
    os.path.join(DATA_DIR, 'generated')
]
# Nơi chứa các file .opt.tour (chuẩn)
optimum_dir = os.path.join(DATA_DIR, 'optimum_solutions')

problems_found = []
print(f"Đang quét tìm các file .tsp có file .opt.tour tương ứng...")

for directory in search_dirs:
    if os.path.isdir(directory):
        # print(f"Đang quét thư mục: {directory}...")
        for filename in os.listdir(directory):
            if filename.endswith(".tsp"):
                problem_name = filename.replace(".tsp", "")
                
                # --- ĐÂY LÀ THAY ĐỔI QUAN TRỌNG ---
                # Kiểm tra xem file .opt.tour có tồn tại không
                opt_path = os.path.join(optimum_dir, f"{problem_name}.opt.tour")
                
                if os.path.exists(opt_path):
                    # Chỉ thêm vào danh sách nếu TÌM THẤY file .opt.tour
                    problems_found.append(problem_name)
                    # print(f"  -> Đã tìm thấy '{problem_name}' (có .opt.tour)")
                else:
                    # Bỏ qua nếu không có
                    # print(f"  -> Bỏ qua '{problem_name}' (không có .opt.tour)")
                    pass
                # --- KẾT THÚC THAY ĐỔI ---
    else:
        print(f"Cảnh báo: Không tìm thấy thư mục {directory}")

# Loại bỏ các file trùng lặp (nếu có) và sắp xếp lại
PROBLEMS_TO_RUN = sorted(list(set(problems_found)))

if not PROBLEMS_TO_RUN:
    print("LỖI: Không tìm thấy file .tsp nào CÓ KÈM file .opt.tour! Dừng thí nghiệm.")
    raise FileNotFoundError("Không tìm thấy bộ dữ liệu hợp lệ (tsp + opt.tour).")
else:
    print(f"\n--- ĐÃ TÌM THẤY {len(PROBLEMS_TO_RUN)} VẤN ĐỀ HỢP LỆ (CÓ .OPT.TOUR) ĐỂ CHẠY ---")
    print(PROBLEMS_TO_RUN)


# --- SIÊU THAM SỐ (HYPERPARAMETERS) --- 
# (Phần này giữ nguyên y hệt như cũ)

SA_PARAMS = {
    # Nhiệt độ ban đầu: Rất cao (10000.0). 
    'initial_temp': 10000.0,
    
    # Tốc độ nguội: Rất chậm (0.999). 
    'cooling_rate': 0.999,
    
    # Nhiệt độ tối thiểu: Điểm dừng.
    'min_temp': 0.1,
    
    # Số lần lặp tại mỗi mức nhiệt:
    'max_iterations_per_temp': 100
}

GA_PARAMS = {
    # Quy mô quần thể: 
    'population_size': 100,
    
    # Số thế hệ:
    'num_generations': 500,
    
    # Tỷ lệ đột biến:
    'mutation_rate': 0.15,
    
    # Kích thước tinh hoa:
    'elite_size': 5,
    
    # Kích thước giải đấu:
    'tournament_k': 5
}

ACO_PARAMS = {
    # Số lượng kiến:
    'num_ants': 50, 
    
    # Số vòng lặp:
    'num_iterations': 100,
    
    # Trọng số Pheromone (alpha):
    'alpha': 1.0, 
    
    # Trọng số Tầm nhìn (beta):
    'beta': 5.0, 
    
    # Tốc độ bay hơi:
    'evaporation_rate': 0.5,
    
    # Lượng Pheromone gửi:
    'pheromone_deposit': 100.0
}

TS_PARAMS = {
    # Số lần lặp tối đa:
    'max_iterations': 2000,
    
    # Nhiệm kỳ cấm (Trí nhớ):
    'tabu_tenure': 30 
}

Đang quét tìm các file .tsp có file .opt.tour tương ứng...

--- ĐÃ TÌM THẤY 31 VẤN ĐỀ HỢP LỆ (CÓ .OPT.TOUR) ĐỂ CHẠY ---
['a280', 'att48', 'bayg29', 'bays29', 'berlin52', 'ch130', 'ch150', 'eil101', 'eil51', 'eil76', 'fri26', 'gr120', 'gr202', 'gr24', 'gr48', 'gr666', 'gr96', 'kroA100', 'kroC100', 'kroD100', 'lin105', 'pa561', 'pcb442', 'pr1002', 'pr2392', 'pr76', 'rd100', 'st70', 'tsp225', 'ulysses16', 'ulysses22']


In [3]:
import pandas as pd
import os

# --- TẢI KẾT QUẢ CŨ ĐỂ KIỂM TRA ---
if os.path.exists(RESULTS_FILE):
    print(f"Đã tìm thấy file kết quả cũ. Đang tải: {RESULTS_FILE}")
    try:
        df_existing = pd.read_csv(RESULTS_FILE)
    except pd.errors.EmptyDataError:
        print("Cảnh báo: File kết quả cũ bị rỗng. Sẽ tạo file mới.")
        df_existing = pd.DataFrame(columns=['problem', 'algorithm'])
else:
    print(f"Không tìm thấy file kết quả cũ. Sẽ tạo file mới: {RESULTS_FILE}")
    df_existing = pd.DataFrame(columns=['problem', 'algorithm'])

# --- *** ĐỊNH NGHĨA HÀM TIỆN ÍCH BỊ THIẾU *** ---
# (Thêm hàm này vào)
def check_if_run_needed(df_existing, problem_name, algo_name):
    """
    Kiểm tra xem một cặp (problem, algorithm) đã tồn tại trong 
    DataFrame kết quả hay chưa.
    """
    if df_existing.empty:
        return True
    
    # Kiểm tra xem có bất kỳ hàng nào khớp với cả hai điều kiện không
    run_exists = ((df_existing['problem'] == problem_name) & 
                  (df_existing['algorithm'] == algo_name)).any()
    
    # Nếu đã tồn tại (run_exists = True) -> Không cần chạy (return False)
    return not run_exists

print("Hàm 'check_if_run_needed' đã được định nghĩa.")

Không tìm thấy file kết quả cũ. Sẽ tạo file mới: ../results/results.csv
Hàm 'check_if_run_needed' đã được định nghĩa.


## 3. Vòng lặp Thí nghiệm Chính

Đây là nơi công việc nặng nhọc diễn ra. Chúng ta lặp qua từng vấn đề, sau đó lặp qua từng thuật toán, ghi lại kết quả.

In [4]:
# Danh sách để lưu tất cả kết quả
all_results = []

# (Giả định 'df_existing' và 'check_if_run_needed' đã được định nghĩa ở cell trên)

print(f"\n--- BẮT ĐẦU THÍ NGHIỆM CHO {len(PROBLEMS_TO_RUN)} VẤN ĐỀ ---")

for problem_name in PROBLEMS_TO_RUN:
    print(f"\n{'='*10} ĐANG XỬ LÝ: {problem_name.upper()} {'='*10}")
    
    # --- 1. TẢI DỮ LIỆU ---
    try:
        coords, matrix = data_loader.load_tsp_problem(problem_name, DATA_DIR)
        if matrix is None: raise ValueError("Không thể tải ma trận.")
        dim = len(matrix)
        opt_tour, opt_cost = data_loader.load_optimum_solution(problem_name, DATA_DIR, matrix)
        
        print(f"Đã tải {problem_name} (N={dim}), Chi phí tối ưu: {opt_cost}")
        
        if opt_cost == 0 and problem_name not in ['test_problem_no_opt']: 
             print(f"CẢNH BÁO NGHIÊM TRỌNG: Chi phí tối ưu = 0. File .opt.tour có thể bị lỗi!")

    except Exception as e:
        print(f"LỖI: Không thể xử lý {problem_name}. Lỗi: {e}. Bỏ qua.")
        continue

    # --- HÀM TRỢ GIÚP GHI KẾT QUẢ ---
    def record_result(algo_name, tour, cost, exec_time):
        if not check_if_run_needed(df_existing, problem_name, algo_name):
            print(f" -> Bỏ qua {algo_name} (đã chạy trước đó).")
            return
        
        gap = evaluator.calculate_gap(cost, opt_cost) # Dùng hàm chuẩn
        
        all_results.append({
            'problem': problem_name,
            'num_cities': dim,
            'algorithm': algo_name,
            'cost': int(cost),
            'opt_cost': int(opt_cost),
            'gap_percent': gap,
            'time_sec': exec_time
        })
        print(f"  [KẾT QUẢ] {algo_name:<20} | Cost: {int(cost):<8} | Gap: {gap:>6.2f}% | Time: {exec_time:>6.3f}s")

    # --- CHẠY 10 THUẬT TOÁN --- 
    
    # --- 1. Nearest Neighbor (ĐÃ SỬA LỖI) --- 
    algo_name = 'Nearest Neighbor'
    if check_if_run_needed(df_existing, problem_name, algo_name):
        print(f"  Đang chạy: {algo_name}...")
        start_time = time.perf_counter()
        
        # SỬA LỖI: Gọi hàm 'solve' đơn giản
        nn_tour, nn_cost = nearest_neighbor.solve(matrix, start_node=0)
                                                
        exec_time = time.perf_counter() - start_time
        record_result(algo_name, nn_tour, nn_cost, exec_time)
    else:
        print(f"  Bỏ qua {algo_name} (đã chạy trước đó).")
        # Vẫn phải chạy lại để lấy tour cho các thuật toán sau
        nn_tour, nn_cost = nearest_neighbor.solve(matrix, start_node=0)

    # --- 2. Nearest Insertion (ĐÃ SỬA LỖI) --- 
    algo_name = 'Nearest Insertion'
    if check_if_run_needed(df_existing, problem_name, algo_name):
        print(f"  Đang chạy: {algo_name}...")
        start_time = time.perf_counter()
        
        # SỬA LỖI: Gọi hàm 'solve' đơn giản
        ni_tour, ni_cost = nearest_insertion.solve(matrix, start_node=0)
                                                 
        exec_time = time.perf_counter() - start_time
        record_result(algo_name, ni_tour, ni_cost, exec_time)

    # --- 3. Christofides --- 
    algo_name = 'Christofides'
    if check_if_run_needed(df_existing, problem_name, algo_name):
        print(f"  Đang chạy: {algo_name}...")
        start_time = time.perf_counter()
        try:
            chris_tour, chris_cost = christofides.solve(matrix)
            exec_time = time.perf_counter() - start_time
            record_result(algo_name, chris_tour, chris_cost, exec_time)
        except Exception as e:
            print(f"  BỎ QUA: Christofides thất bại (Lỗi: {e})")
    
    # --- 4. 2-Opt (từ kết quả NN) --- 
    algo_name = '2-Opt' 
    if check_if_run_needed(df_existing, problem_name, algo_name):
        print(f"  Đang chạy: {algo_name} (from NN)...")
        start_time = time.perf_counter()
        
        # 2-Opt nhận 'nn_tour' (chưa tối ưu)
        two_opt_tour, two_opt_cost = two_opt.solve(matrix, initial_tour=nn_tour) 
        
        exec_time = time.perf_counter() - start_time
        record_result(algo_name, two_opt_tour, two_opt_cost, exec_time)

    # --- 5. Simulated Annealing (từ kết quả NN) --- 
    algo_name = 'Simulated Annealing'
    if check_if_run_needed(df_existing, problem_name, algo_name):
        print(f"  Đang chạy: {algo_name} (from NN)...")
        start_time = time.perf_counter()
        sa_tour, sa_cost = simulated_annealing.solve(matrix, initial_tour=nn_tour, **SA_PARAMS)
        exec_time = time.perf_counter() - start_time
        record_result(algo_name, sa_tour, sa_cost, exec_time)

    # --- 6. Genetic Algorithm --- 
    algo_name = 'Genetic Algorithm'
    if check_if_run_needed(df_existing, problem_name, algo_name):
        print(f"  Đang chạy: {algo_name}...")
        start_time = time.perf_counter()
        ga_tour, ga_cost = genetic_algorithm.solve(matrix, **GA_PARAMS)
        exec_time = time.perf_counter() - start_time
        record_result(algo_name, ga_tour, ga_cost, exec_time)

    # --- 7. Ant Colony Optimization --- 
    algo_name = 'ACO'
    if check_if_run_needed(df_existing, problem_name, algo_name):
        print(f"  Đang chạy: {algo_name}...")
        start_time = time.perf_counter()
        aco_tour, aco_cost = ant_colony.solve(matrix, **ACO_PARAMS)
        exec_time = time.perf_counter() - start_time
        record_result(algo_name, aco_tour, aco_cost, exec_time)

    # --- 8. Tabu Search (từ kết quả NN) --- 
    algo_name = 'Tabu Search'
    if check_if_run_needed(df_existing, problem_name, algo_name):
        print(f"  Đang chạy: {algo_name} (from NN)...")
        start_time = time.perf_counter()
        ts_tour, ts_cost = tabu_search.solve(matrix, initial_tour=nn_tour, **TS_PARAMS)
        exec_time = time.perf_counter() - start_time
        record_result(algo_name, ts_tour, ts_cost, exec_time)

    # --- 9. Brute Force (Giữ nguyên) --- 
    algo_name = 'Brute Force'
    if dim <= 10: 
        if check_if_run_needed(df_existing, problem_name, algo_name):
            print(f"  Đang chạy: {algo_name} (N<=10)...")
            start_time = time.perf_counter()
            bf_tour, bf_cost = brute_force.solve(matrix)
            exec_time = time.perf_counter() - start_time
            record_result(algo_name, bf_tour, bf_cost, exec_time)
    else:
        print(f"  BỎ QUA: Brute Force (N={dim} > 10)")

    # --- 10. Held-Karp (Giữ nguyên) --- 
    algo_name = 'Held-Karp'
    if dim <= 21: 
        if check_if_run_needed(df_existing, problem_name, algo_name):
            print(f"  Đang chạy: {algo_name} (N<=21)...")
            start_time = time.perf_counter()
            hk_tour, hk_cost = held_karp.solve(matrix)
            exec_time = time.perf_counter() - start_time
            record_result(algo_name, hk_tour, hk_cost, exec_time)
    else:
        print(f"  BỎ QUA: Held-Karp (N={dim} > 21)")
        
    print(f"--- Hoàn thành {problem_name.upper()} ---\n")

print("\n--- TẤT CẢ THÍ NGHIỆM HOÀN TẤT ---\n")


--- BẮT ĐẦU THÍ NGHIỆM CHO 31 VẤN ĐỀ ---

========== ĐANG XỬ LÝ: A280 ==========
Đã tải a280 (N=280), Chi phí tối ưu: 2579
  Đang chạy: Nearest Neighbor...
  [KẾT QUẢ] Nearest Neighbor     | Cost: 3157     | Gap:  22.41% | Time:  0.001s
  Đang chạy: Nearest Insertion...


  [KẾT QUẢ] Nearest Insertion    | Cost: 3094     | Gap:  19.97% | Time:  0.046s
  Đang chạy: Christofides...
  [KẾT QUẢ] Christofides         | Cost: 2914     | Gap:  12.99% | Time:  0.622s
  Đang chạy: 2-Opt (from NN)...
  [KẾT QUẢ] 2-Opt                | Cost: 2767     | Gap:   7.29% | Time:  0.668s
  Đang chạy: Simulated Annealing (from NN)...
  [KẾT QUẢ] Simulated Annealing  | Cost: 2853     | Gap:  10.62% | Time:  5.642s
  Đang chạy: Genetic Algorithm...
[Gen   1] Best=  3109, Current=  3109, Diversity=100.0%, MutRate=15.0%, NoImprove=0
[Gen   2] Best=  3109, Current=  3109, Diversity=97.0%, MutRate=15.0%, NoImprove=1
[Gen   3] Best=  3109, Current=  3109, Diversity=99.0%, MutRate=15.0%, NoImprove=2
[Gen   4] Best=  3109, Current=  3109, Diversity=96.0%, MutRate=15.0%, NoImprove=3
[Gen   5] Best=  3109, Current=  3109, Diversity=92.0%, MutRate=15.0%, NoImprove=4
[Gen  50] Best=  3102, Current=  3102, Diversity=33.0%, MutRate=30.0%, NoImprove=42
[Gen 54] Diversity 24.0% < threshol

## 4. Xử lý, Hiển thị và Lưu Kết quả

Bây giờ chúng ta chuyển danh sách kết quả (list of dicts) thành một DataFrame `pandas` đẹp đẽ, tính toán `gap %` và lưu vào file `results.csv`.

In [5]:
import pandas as pd

# --- XEM LẠI KẾT QUẢ MỚI VỪA CHẠY ---
print("--- KẾT QUẢ MỚI TỪ LẦN CHẠY NÀY ---")
df_new = pd.DataFrame(all_results)

if not df_new.empty:
    # Sắp xếp lại cột cho df_new (đảm bảo tên khớp với vòng lặp)
    try:
        columns_order = [
            'problem', 'num_cities', 'algorithm', 
            'cost', 'opt_cost', 'gap_percent', 'time_sec'
        ]
        # Hiển thị tất cả kết quả mới
        print(df_new[columns_order].to_string())
    except KeyError:
        # Xử lý nếu tên cột không khớp (ví dụ: 'dimension' thay vì 'num_cities')
        print("Lỗi sắp xếp cột (KeyError), đang in kết quả thô:")
        print(df_new.to_string())
else:
    print("Không có kết quả mới nào được ghi nhận (danh sách 'all_results' rỗng).")


# --- 6. GỘP VÀ LƯU KẾT QUẢ VÀO FILE CSV (LOGIC AN TOÀN) ---
print("\n--- 6. LƯU KẾT QUẢ VÀO FILE CSV ---")
print("Đang gộp kết quả mới và cũ...")

# (Giả định 'df_existing' đã được tải ở cell phía trên)
df_final = pd.concat([df_existing, df_new], ignore_index=True)

# Loại bỏ các bản ghi trùng lặp, giữ lại bản ghi cuối cùng (mới nhất)
df_final = df_final.drop_duplicates(subset=['problem', 'algorithm'], keep='last')

# Sắp xếp lại
df_final = df_final.sort_values(by=['problem', 'algorithm'])

# Lưu file
try:
    df_final.to_csv(RESULTS_FILE, index=False)
    print(f"\nĐÃ LƯU KẾT QUẢ! Tổng cộng {len(df_final)} hàng trong {RESULTS_FILE}")
except Exception as e:
    print(f"\nLỖI: Không thể lưu file CSV. Lỗi: {e}")

# Hiển thị 5 hàng cuối cùng của file TỔNG
print("\n--- 5 hàng cuối cùng trong file CSV tổng ---")
# Sử dụng display() thay vì print() để Jupyter Notebook hiển thị bảng đẹp
display(df_final.tail(5))

--- KẾT QUẢ MỚI TỪ LẦN CHẠY NÀY ---
       problem  num_cities            algorithm    cost  opt_cost  gap_percent     time_sec
0         a280         280     Nearest Neighbor    3157      2579       22.412     0.001327
1         a280         280    Nearest Insertion    3094      2579       19.969     0.046191
2         a280         280         Christofides    2914      2579       12.990     0.621953
3         a280         280                2-Opt    2767      2579        7.290     0.667864
4         a280         280  Simulated Annealing    2853      2579       10.624     5.642060
5         a280         280    Genetic Algorithm    2915      2579       13.028    40.972215
6         a280         280                  ACO    3077      2579       19.310    35.239364
7         a280         280          Tabu Search    2730      2579        5.855    80.401165
8        att48          48     Nearest Neighbor   12832     10598       21.079     0.000217
9        att48          48    Nearest Insert

,problem,algorithm,num_cities,cost,opt_cost,gap_percent,time_sec
246,ulysses22,Genetic Algorithm,22.0,6958.0,6971.0,-0.186,1.583581
242,ulysses22,Nearest Insertion,22.0,7642.0,6971.0,9.626,0.000572
241,ulysses22,Nearest Neighbor,22.0,8155.0,6971.0,16.985,0.000146
245,ulysses22,Simulated Annealing,22.0,6958.0,6971.0,-0.186,5.160627
248,ulysses22,Tabu Search,22.0,6958.0,6971.0,-0.186,0.406516


## 5. Phân tích Nhanh

Hãy nhóm theo thuật toán và tính toán trung bình `gap_percent` và `time`.

In [6]:
# Sửa lỗi: Thay thế df_results bằng df_final (chứa tất cả kết quả đã lưu)

# Đảm bảo bạn đã chạy cell lưu trữ và df_final đã được tạo thành công
if not df_final.empty:
    print("\n--- PHÂN TÍCH NHANH (TRUNG BÌNH) ---")
    
    # Chúng ta chỉ tính trung bình cho các vấn đề Heuristic (N > 21)
    # Tên cột trong df_final là 'num_cities'
    heuristic_results = df_final[df_final['num_cities'] > 21]

    if not heuristic_results.empty:
        # Tên cột trong df_final là 'time_sec'
        analysis = heuristic_results.groupby('algorithm')[['gap_percent', 'time_sec']].mean()
        analysis = analysis.sort_values(by='gap_percent')
        
        print("\nBảng Phân tích Trung bình (N > 21):")
        print(analysis)
        
    else:
        print("Không có kết quả từ các vấn đề lớn (N>21) để phân tích.")
else:
    print("Không có kết quả nào được tạo ra.")


--- PHÂN TÍCH NHANH (TRUNG BÌNH) ---

Bảng Phân tích Trung bình (N > 21):
                     gap_percent    time_sec
algorithm                                   
Tabu Search             2.388200  279.617422
Simulated Annealing     5.398433    5.262776
2-Opt                   6.901967    1.348249
Genetic Algorithm       7.763167  119.398484
ACO                     8.846467   37.665931
Christofides           11.489333   11.436591
Nearest Insertion      18.375967    0.888203
Nearest Neighbor       24.541967    0.001466
